In [6]:
import yfinance as yf
import pandas as pd
import random
from collections import defaultdict

In [12]:
def fetch_yahoo(symbol="^NSEI", interval="1m", period="5d"):
    df = yf.download(
        tickers=symbol,
        interval=interval,
        period=period,
        auto_adjust=True,
        progress=False
    )
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.get_level_values(0)

    df = df.reset_index()

    # 🔥 FORCE timestamp to pandas datetime
    df["timestamp"] = pd.to_datetime(df["Datetime"], utc=True)

    # 🔥 Convert to integer nanoseconds (HFT-style)
    df["ts_ns"] = df["timestamp"].astype("int64")

    df.dropna(inplace=True)
    return df

def generate_l2_snapshot(mid, spread=0.5, levels=5):
    bids, asks = {}, {}

    for i in range(1, levels + 1):
        bids[round(mid - i * spread, 2)] = random.randint(50, 300)
        asks[round(mid + i * spread, 2)] = random.randint(50, 300)

    return bids, asks

def simulate_nse_ticks(df):
    events = []

    for _, row in df.iterrows():
        ts = int(row["ts_ns"])

        high = float(row["High"])
        low = float(row["Low"])
        mid = (high + low) / 2.0

        bids, asks = generate_l2_snapshot(mid)

        # L2 book init
        for p, q in bids.items():
            events.append((ts, "BUY", float(p), int(q), "NEW"))

        for p, q in asks.items():
            events.append((ts, "SELL", float(p), int(q), "NEW"))

        trade_count = max(5, min(int(row["Volume"] // 1000), 20))

        for _ in range(trade_count):
            side = random.choice(["BUY", "SELL"])
            price = (
                random.choice(list(asks.keys()))
                if side == "BUY"
                else random.choice(list(bids.keys()))
            )
            qty = random.randint(10, 50)

            events.append((ts, side, float(price), qty, "TRADE"))

        # Optional cancel events
        if random.random() < 0.3:
            side = random.choice(["BUY", "SELL"])
            price = mid
            qty = random.randint(10, 50)
            events.append((ts, side, float(price), qty, "CANCEL"))

    # 🔥 SAFE SORT
    return sorted(events, key=lambda x: int(x[0]))

class OrderBook:
    def __init__(self):
        self.bids = defaultdict(int)
        self.asks = defaultdict(int)
    def process_events(self,event):
        ts,side,price,qty,etype = event
    
        book = self.bids if side == 'BUY' else self.asks

        if etype == 'NEW':
            book[price] = book.get(price,0) + qty

        elif etype == 'CANCEL':
            if price in book:
                book[price] =- qty
                if book[price] <= 0:
                    del book[price]
        elif etype == 'TRADE':
            if price in book:
                book[price] =- qty
                if book[price] <= 0:
                    del book[price]

def imbalance(book, depth = 3):
    bid_vol = sum(book.bids[p] for p in sorted(book.bids, reverse = True)[:depth])
    ask_vol = sum(book.asks[p] for p in sorted(book.asks)[:depth])

    if bid_vol + ask_vol == 0:
        return 0.5
    else:
        return bid_vol / (bid_vol + ask_vol)


        

In [13]:
def run_backtest(events):
    book = OrderBook()
    position = 0
    entry = 0
    pnl = 0

    for event in events:
        book.process_events(event)
        imb = imbalance(book)
        price = event[2]

        if imb > 0.7 and position == 0:
            position = 1
            entry = price

        elif imb < 0.3 and position == 1:
            pnl += price - entry
            position = 0

    return pnl

if __name__ == "__main__":
    df = fetch_yahoo("^NSEI", interval="1m", period="5d")
    events = simulate_nse_ticks(df)
    pnl = run_backtest(events)

    print("Total PnL:", pnl)


Total PnL: 268.3600000000006
